In [ ]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# шлях до тренувальних справжніх даних
real_path = ""
# шлях до тренувальних синтезованих даних
fake_path = ""
real = []
fake = []

vreal_path = ""
vfake_path = ""
vreal = []
vfake = []

def get_mel_spectrogram(folder_path):
    mel_specs = []
    for filename in os.listdir(folder_path):
        audio, sr = librosa.load(os.path.join(folder_path, filename), sr=None)
        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        mel_specs.append(mel_spec_db)
    return mel_specs

real = get_mel_spectrogram(real_path)
fake = get_mel_spectrogram(fake_path)

vreal = get_mel_spectrogram(vreal_path)
vfake = get_mel_spectrogram(vfake_path)


In [ ]:
print("Кількість тренувальних аудіофайлів real:", len(real))
print("Кількість тренувальних аудіофайлів fake:", len(fake))

print("Кількість валідаційних аудіофайлів real:", len(vreal))
print("Кількість валідаційних аудіофайлів fake:", len(vfake))

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

real_labels = np.zeros(len(real))
fake_labels = np.ones(len(fake))

vreal_labels = np.zeros(len(vreal))
vfake_labels = np.ones(len(vfake))

X_train = np.concatenate((real, fake), axis=0)
y_train = np.concatenate((real_labels, fake_labels), axis=0)

X_val = np.concatenate((vreal, vfake), axis=0)
y_val = np.concatenate((vreal_labels, vfake_labels), axis=0)

shuffle_index_train = np.random.permutation(len(X_train))
X_train_shuffled = X_train[shuffle_index_train]
y_train_shuffled = y_train[shuffle_index_train]

shuffle_index_val = np.random.permutation(len(X_val))
X_val_shuffled = X_val[shuffle_index_val]
y_val_shuffled = y_val[shuffle_index_val]

print("Розмірність тренувальних даних:", X_train_shuffled.shape)
print("Розмірність тестових даних:", X_val_shuffled.shape)
print("Розмірність міток тренувальних даних:", y_train_shuffled.shape)
print("Розмірність міток тестових даних:", y_val_shuffled.shape)

X_train = X_train_shuffled[..., np.newaxis]
X_test = X_val_shuffled[..., np.newaxis]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1:])),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val_shuffled, y_val_shuffled))


In [ ]:
model.save("CNN_Mel_Spec_DB.keras")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val_shuffled, y_val_shuffled))

In [ ]:
model.save("LSTM_Mel_Spec_DB.keras")